# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 9 2022 11:56AM,250408,19,8754033,"Emersa Waterbox, LLC",Released
1,Nov 9 2022 11:46AM,250407,10,CTF0011287-BO,"Citieffe, Inc.",Released
2,Nov 9 2022 11:43AM,250405,10,CTF0011372-BO,"Citieffe, Inc.",Released
3,Nov 9 2022 11:40AM,250404,10,CTF0011461-BO,"Citieffe, Inc.",Released
4,Nov 9 2022 11:13AM,250400,10,MSP215657,"Methapharm, Inc.",Executing
5,Nov 9 2022 11:13AM,250400,10,MSP215659,"Methapharm, Inc.",Executing
6,Nov 9 2022 11:13AM,250400,10,MSP215680,Methapharm-G,Released
7,Nov 9 2022 11:13AM,250400,10,MSP215687,"Methapharm, Inc.",Executing
8,Nov 9 2022 11:01AM,250399,10,CTF0011496-BO,"Citieffe, Inc.",Released
9,Nov 9 2022 10:30AM,250394,10,8749616,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,250400,Released,1
30,250404,Released,1
31,250405,Released,1
32,250407,Released,1
33,250408,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250400,NaN,3.0,1.0
250404,NaN,NaN,1.0
250405,NaN,NaN,1.0
250407,NaN,NaN,1.0
250408,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250318,69.0,7.0,14.0
250338,0.0,0.0,1.0
250341,0.0,1.0,0.0
250356,0.0,0.0,1.0
250357,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250318,69,7,14
250338,0,0,1
250341,0,1,0
250356,0,0,1
250357,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250318,69,7,14
1,250338,0,0,1
2,250341,0,1,0
3,250356,0,0,1
4,250357,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250318,69,7,14
1,250338,,,1
2,250341,,1,
3,250356,,,1
4,250357,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 9 2022 11:56AM,250408,19,"Emersa Waterbox, LLC"
1,Nov 9 2022 11:46AM,250407,10,"Citieffe, Inc."
2,Nov 9 2022 11:43AM,250405,10,"Citieffe, Inc."
3,Nov 9 2022 11:40AM,250404,10,"Citieffe, Inc."
4,Nov 9 2022 11:13AM,250400,10,"Methapharm, Inc."
6,Nov 9 2022 11:13AM,250400,10,Methapharm-G
8,Nov 9 2022 11:01AM,250399,10,"Citieffe, Inc."
9,Nov 9 2022 10:30AM,250394,10,Eywa Pharma Inc.
26,Nov 9 2022 10:27AM,250393,10,Eywa Pharma Inc.
29,Nov 9 2022 10:26AM,250382,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 9 2022 11:56AM,250408,19,"Emersa Waterbox, LLC",,,1
1,Nov 9 2022 11:46AM,250407,10,"Citieffe, Inc.",,,1
2,Nov 9 2022 11:43AM,250405,10,"Citieffe, Inc.",,,1
3,Nov 9 2022 11:40AM,250404,10,"Citieffe, Inc.",,,1
4,Nov 9 2022 11:13AM,250400,10,"Methapharm, Inc.",,3,1
5,Nov 9 2022 11:13AM,250400,10,Methapharm-G,,3,1
6,Nov 9 2022 11:01AM,250399,10,"Citieffe, Inc.",,,1
7,Nov 9 2022 10:30AM,250394,10,Eywa Pharma Inc.,,5,12
8,Nov 9 2022 10:27AM,250393,10,Eywa Pharma Inc.,,,3
9,Nov 9 2022 10:26AM,250382,10,Bio-PRF,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 11:56AM,250408,19,"Emersa Waterbox, LLC",1,,
1,Nov 9 2022 11:46AM,250407,10,"Citieffe, Inc.",1,,
2,Nov 9 2022 11:43AM,250405,10,"Citieffe, Inc.",1,,
3,Nov 9 2022 11:40AM,250404,10,"Citieffe, Inc.",1,,
4,Nov 9 2022 11:13AM,250400,10,"Methapharm, Inc.",1,3,
5,Nov 9 2022 11:13AM,250400,10,Methapharm-G,1,3,
6,Nov 9 2022 11:01AM,250399,10,"Citieffe, Inc.",1,,
7,Nov 9 2022 10:30AM,250394,10,Eywa Pharma Inc.,12,5,
8,Nov 9 2022 10:27AM,250393,10,Eywa Pharma Inc.,3,,
9,Nov 9 2022 10:26AM,250382,10,Bio-PRF,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 11:56AM,250408,19,"Emersa Waterbox, LLC",1,,
1,Nov 9 2022 11:46AM,250407,10,"Citieffe, Inc.",1,,
2,Nov 9 2022 11:43AM,250405,10,"Citieffe, Inc.",1,,
3,Nov 9 2022 11:40AM,250404,10,"Citieffe, Inc.",1,,
4,Nov 9 2022 11:13AM,250400,10,"Methapharm, Inc.",1,3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 11:56AM,250408,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
1,Nov 9 2022 11:46AM,250407,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Nov 9 2022 11:43AM,250405,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Nov 9 2022 11:40AM,250404,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Nov 9 2022 11:13AM,250400,10,"Methapharm, Inc.",1.0,3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 9 2022 11:56AM,250408,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
1,Nov 9 2022 11:46AM,250407,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Nov 9 2022 11:43AM,250405,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Nov 9 2022 11:40AM,250404,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Nov 9 2022 11:13AM,250400,10,"Methapharm, Inc.",1.0,3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3255105,49.0,20.0,10.0
15,500682,24.0,11.0,69.0
16,751051,3.0,0.0,0.0
19,250408,1.0,0.0,0.0
21,2253411,8.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3255105,49.0,20.0,10.0
1,15,500682,24.0,11.0,69.0
2,16,751051,3.0,0.0,0.0
3,19,250408,1.0,0.0,0.0
4,21,2253411,8.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,49.0,20.0,10.0
1,15,24.0,11.0,69.0
2,16,3.0,0.0,0.0
3,19,1.0,0.0,0.0
4,21,8.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,49.0
1,15,Released,24.0
2,16,Released,3.0
3,19,Released,1.0
4,21,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,10.0,69.0,0.0,0.0,0.0
Executing,20.0,11.0,0.0,0.0,1.0
Released,49.0,24.0,3.0,1.0,8.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,10.0,69.0,0.0,0.0,0.0
1,Executing,20.0,11.0,0.0,0.0,1.0
2,Released,49.0,24.0,3.0,1.0,8.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,10.0,69.0,0.0,0.0,0.0
1,Executing,20.0,11.0,0.0,0.0,1.0
2,Released,49.0,24.0,3.0,1.0,8.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()